In [1]:
###############################################################################
# Import packages
###############################################################################
import os
import time
from tqdm import tqdm
import random 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mglearn
from scipy.spatial.distance import cdist
import gurobipy as gp
from gurobipy import GRB
from scipy.stats import skew
import math
import itertools
from sklearn.cluster import DBSCAN

# Load functions
from helperfunctions_data_generation import distance, fun_convert_time, fun_save_file, fun_load_file
from helperfunctions_data_generation import plot_instance
from helperfunctions_data_generation import fun_multi_dbscan, fun_cluster_features

In [ ]:
# Assign string 'TSP' or 'CVRP' to the following variable to define the routing problem
routing_problem = 'TSP'
if (routing_problem == 'TSP'):
    folder = '01_TSP'
    file_name = 'tsp_instances_j.xlsx'
elif (routing_problem == 'CVRP'):
    folder = '02_CVRP'
    file_name = 'cvrp_instances_j.xlsx'

# Load data
data_original = fun_load_file(subfolder_path='..\\..\\01_data\\' + folder, name=file_name)

In [3]:
# Remove first example instance before selecting and modifying columns
#print('Columns:', data_original.columns)
data_original = data_original[data_original['Instance_id'] != 0].reset_index(drop = True)
data = data_original[['X', 'Y', 'Marginal_Cost', 'Savings', 'Shapley Value', 'Total cost', 'Number Customers', 'X_Depot', 'Y_Depot', 'Instance_id', 'SHAPO', 'Percentage_error']]
data = data.rename(columns={'Marginal_Cost': 'Marginal Costs', 'X_Depot': 'X Depot', 'Y_Depot': 'Y Depot', 'Total cost': 'Total Costs', 
                            'Instance_id': 'Instance ID', 'Percentage_error': 'Percentage Error'})

# Reorder columns 
columns = ['Instance ID', 'Number Customers', 'X', 'Y', 'X Depot', 'Y Depot', 'Savings', 'Marginal Costs', 'Total Costs', 'Shapley Value', 'SHAPO', 'Percentage Error']
if (routing_problem == 'CVRP'): columns += ['Vehicle Capacity', 'Demand', 'Total Demand']
data = data[columns]

# Print number of instances
print('Number of instances:', len(np.unique(data['Instance ID'])))

# Get a list of all instance sizes and the number of instances per size
num_customers_list = np.unique(data['Number Customers'])
number_of_instances_per_size = int(len(data[data['Number Customers'] == num_customers_list[0]]) / num_customers_list[0])
print('Instance sizes: {}\nNumber of instances per size: {}'.format(num_customers_list, number_of_instances_per_size))
display(data)

# Create a DataFrame with previous cluster features of Johannes
cluster_features_j = ['cluster', 'customers_in_same_cluster', 'distance_to_cluster_center', # Feature distance_to_cluster_center_ratio fehlt im Datensatz
                      'min_distance_to_other_cluster', 'min_distance_to_other_cluster_ratio', 'cluster_distance_to_depot', 'cluster_distance_to_depot_ratio', 
                      'Cluster Area', 'Cluster Area ratio']
df_clusters_johannes = data_original[cluster_features_j]
df_clusters_johannes.columns = [string + str('_j') for string in cluster_features_j]
df_clusters_johannes

Number of instances: 9000
Instance sizes: [ 6  7  8  9 10 11 12 13 14]
Number of instances per size: 1000


,Instance ID,Number Customers,X,Y,X Depot,Y Depot,Savings,Marginal Costs,Total Costs,Shapley Value,SHAPO,Percentage Error
0,1,6,11.757432,50.848731,2.380844,66.016752,0.318205,0.318205,227.291186,6.805996,6.847093,0.006038
1,1,6,83.228495,41.537025,2.380844,66.016752,30.168031,34.871270,227.291186,73.361446,72.126416,0.016835
2,1,6,33.032921,29.876631,2.380844,66.016752,2.596858,2.596858,227.291186,21.568006,21.751330,0.008500
3,1,6,42.131509,30.755973,2.380844,66.016752,7.186744,7.186744,227.291186,25.980268,26.403603,0.016294
4,1,6,54.103013,58.267699,2.380844,66.016752,1.415778,1.415778,227.291186,30.191952,30.346193,0.005109
...,...,...,...,...,...,...,...,...,...,...,...,...
89995,9000,14,35.411268,13.512220,83.308855,83.076767,4.147751,4.147751,370.421950,32.946966,33.551630,0.018353
89996,9000,14,94.027698,4.059342,83.308855,83.076767,49.854239,49.854239,370.421950,70.014213,69.654750,0.005134
89997,9000,14,15.703090,62.406463,83.308855,83.076767,29.195471,34.447352,370.421950,37.349798,42.540654,0.138979
89998,9000,14,46.611090,57.482706,83.308855,83.076767,1.835545,14.053805,370.421950,16.911138,12.089902,0.285092


,cluster_j,customers_in_same_cluster_j,distance_to_cluster_center_j,min_distance_to_other_cluster_j,min_distance_to_other_cluster_ratio_j,cluster_distance_to_depot_j,cluster_distance_to_depot_ratio_j,Cluster Area_j,Cluster Area ratio_j
0,2,1,8.916127,29.874328,0.827656,8.916127,0.165161,0.000000,0.000000
1,1,2,16.794412,42.487567,1.177101,68.215562,1.263612,848.293692,1.360991
2,0,3,12.132511,29.874328,0.827656,59.520110,1.102539,888.816789,1.426006
3,0,3,10.321964,30.003533,0.831236,59.520110,1.102539,888.816789,1.426006
4,1,2,16.794412,30.003533,0.831236,68.215562,1.263612,848.293692,1.360991
...,...,...,...,...,...,...,...,...,...
89995,1,4,19.362833,29.590372,0.645451,78.087032,1.454288,3435.885922,1.481249
89996,1,4,40.011040,56.598097,1.234568,78.087032,1.454288,3435.885922,1.481249
89997,0,9,22.883805,47.383902,1.033580,47.813029,0.890467,2338.918164,1.008334
89998,0,9,14.494172,38.366084,0.836875,47.813029,0.890467,2338.918164,1.008334


In [2]:
###############################################################################
# Generate random instances - settings
###############################################################################
# Define the range for X and Y coordinates and create DataFrame to merge final instances
x_range = (0, 100)
y_range = (0, 100)
df_final = pd.DataFrame()
run_time = {}
prints = False # Show prints and interim results during generating
max_instance_id = np.inf # Set parameter to 'np.inf' to create features for all instances
old_cluster_features = False # Parameter to add previous cluster features of Johannes or not

###############################################################################
# START GENERAITING
###############################################################################
for instance_id in tqdm(np.unique(data['Instance ID']), desc='Processing', unit='iteration', leave=True):

    if (instance_id % number_of_instances_per_size == 1): start = time.time() # Start time count if a new instance sice starts
    instance_df = data[data['Instance ID'] == instance_id]
    num_customers = int(instance_df.iloc[0]['Number Customers'])

    # Add depot with X and Y coordinates as first row (like in the original setting from function generate_random_instance_df)
    x_depot = instance_df.iloc[0]['X Depot'] # X coordinate of depot
    y_depot = instance_df.iloc[0]['Y Depot'] # Y coordinate of depot
    df_depot = pd.DataFrame(data={'Instance ID': instance_id, 'Number Customers': num_customers, 
                                  'X': x_depot, 'Y': y_depot}, index=[0])
    instance_df = pd.concat([df_depot, instance_df], ignore_index=True)[columns]

    if (prints == True): print('############### INSTANCE ###############'); display(instance_df)
    
    ###############################################################################
    # Add distance features
    ###############################################################################

    # Calculate the distance between the depot and each location
    instance_df['Depot Distance'] = np.sqrt((instance_df['X'] - x_depot) ** 2 + (instance_df['Y'] - y_depot) ** 2) # Euclidean distance

    # Calculate distances from each customer to all other customers (excluding depot) - customer distances matrix
    customer_distances = cdist(instance_df.iloc[1:][['X', 'Y']], instance_df.iloc[1:][['X', 'Y']], metric='euclidean')

    # Replace diagonal values with np.nan; otherwise the customer will be his own closest customer
    np.fill_diagonal(customer_distances, np.nan)

    # Create an array with indices of all customers and loop over each of them (excluding the depot)
    all_customers = np.arange(start=1, stop=num_customers+1)
    for i, customer in enumerate(all_customers): # i starts with zero, customer starts with one

        # Sort the distances for the current customer
        sorted_distances = np.sort(customer_distances[i])

        # Add columns for the distances to the closest, second closest, third closest, and fourth closest customers and add the respective values of the customer
        instance_df.loc[customer, 'Closest Customer Distance (CCD)'] = sorted_distances[0]
        instance_df.loc[customer, '2nd CCD'] = sorted_distances[1]
        instance_df.loc[customer, '3rd CCD'] = sorted_distances[2]
        instance_df.loc[customer, '4th CCD'] = sorted_distances[3]
        instance_df.loc[customer, '5th CCD'] = sorted_distances[4]
        instance_df.loc[customer, '6th CCD'] = sorted_distances[5] if (num_customers >= 7) else 100000
        instance_df.loc[customer, '7th CCD'] = sorted_distances[6] if (num_customers >= 8) else 100000
        instance_df.loc[customer, '8th CCD'] = sorted_distances[7] if (num_customers >= 9) else 100000
        instance_df.loc[customer, '9th CCD'] = sorted_distances[8] if (num_customers >= 10) else 100000
        instance_df.loc[customer, '10th CCD'] = sorted_distances[9] if (num_customers >= 11) else 100000

    # Calculate the mean distance to other customers
    customer_distances = pd.DataFrame(customer_distances, index=all_customers, columns=all_customers)
    mean_distances = customer_distances.mean(axis=1, skipna=True) # Calculate mean for each row over all columns, ignoring the np.nan

    # Add the 'Mean distance to other customers' column to the DataFrame (We add a NaN value for the depot since it doesn't have a mean distance)
    instance_df['Mean Distance To Other Customers'] = [np.nan] + mean_distances.tolist()
    
    # Calculate for each customer the distance to centroid of all customrs (excluding the depot) and save it in a new column
    centroid_x = instance_df.loc[1:, 'X'].mean()
    centroid_y = instance_df.loc[1:, 'Y'].mean()
    instance_df['Gravity Center Distance'] = np.sqrt((instance_df.loc[1:, 'X'] - centroid_x) ** 2 + (instance_df.loc[1:, 'Y'] - centroid_y) ** 2)
    
    if (prints == True):
        print('\n############### DISTANCE FEATURES ###############')
        print('Customer distances:'); display(customer_distances, instance_df)

    ###############################################################################
    # Add cluster features
    ###############################################################################

    # Cluster customers (without depot)
    # Get X and Y coordinates of the customers (excluding the depot)
    X = instance_df[['X', 'Y']][1:]

    # Clustering the customers of the instance
    assignments, core_point_indices = fun_multi_dbscan(X=X, num_customers=num_customers, prints=prints)

    # Extract X and Y coordinates of depot and all customers (first two columns) as tuples and give it as input for plot_instance
    coord = [(instance_df['X'][i], instance_df['Y'][i]) for i in range(len(instance_df))]
    
    # View clusters
    if (prints == True):
        print('\n############### CLUSTER CUSTOMERS ###############')
        plot_instance(coord=coord, sequence=None, total_costs=None, x_range=x_range, y_range=y_range, 
                 assignments=assignments, core_point_indices=core_point_indices, plot_sequence=False)

    # Define columns for displaying the instance
    instance_features = ['Instance ID', 'Number Customers', 'X', 'Y', 'X Depot', 'Y Depot']
    cluster_features = ['Cluster', 'Core Point', 'Outlier', 'Number Clusters', 'Number Outliers', 'Cluster Size', 'X Centroid', 'Y Centroid', 'Centroid Distance', 
                        'Centroid Distance To Depot', 'Distance To Closest Other Cluster', 'Distance To Closest Other Centroid', 'Cluster Area', 'Cluster Density']

    # Execute function for cluster features
    instance_df = fun_cluster_features(data=instance_df, assignments=assignments, core_point_indices=core_point_indices, 
                                        features=instance_features+cluster_features, prints=prints)
    
    ###############################################################################
    # Add statistics as features
    ###############################################################################

    # Extract X and Y values of the customers, excluding the first row (depot)
    x_customers = instance_df['X'].iloc[1:]
    y_customers = instance_df['Y'].iloc[1:]

    # Compute statistics for X and Y coordinates of the customers and add each of them as a column
    for statistic in [np.mean, np.std, np.max, np.min]:
        instance_df['X ' + statistic.__name__.capitalize()] = statistic(x_customers)
        instance_df['Y ' + statistic.__name__.capitalize()] = statistic(y_customers)

    # Calculate the correlation between X and Y of the customers as well as the skewness for X and Y and add statistics to each row as a column
    instance_df['Correlation'] = np.corrcoef(x_customers, y_customers)[0, 1] # Select first row and second column of correlation matrix
    instance_df['Skewness X'] = skew(x_customers)
    instance_df['Skewness Y'] = skew(y_customers)

    # Reorder columns
    last_columns = ['Savings', 'Marginal Costs', 'Total Costs', 'Shapley Value', 'SHAPO', 'Percentage Error']
    instance_df = instance_df[[column for column in instance_df.columns if column not in last_columns] + last_columns]

    # Define columns for displaying the instance
    statistical_features = ['X Mean', 'Y Mean', 'X Std', 'Y Std', 'X Max', 'Y Max', 'X Min', 'Y Min', 'Correlation', 'Skewness X', 'Skewness Y']

    if (prints == True): print('\n############### STATISTICAL FEATURES ###############'); display(instance_df[instance_features + statistical_features])

    ##################################################################################################################################################################
    # Solve the instance here and add features Savings, Marginal Costs, Total Costs (and Shapley Value)

    
    ##################################################################################################################################################################

    ###############################################################################
    # Add ratio features
    ###############################################################################

    # Define features for which the ratio should be computed
    ratio_features = ['Depot Distance', 'Closest Customer Distance (CCD)', '2nd CCD', '3rd CCD', '4th CCD', '5th CCD', '6th CCD', '7th CCD', '8th CCD', '9th CCD', '10th CCD',
                      'Mean Distance To Other Customers', 'Gravity Center Distance', 'Centroid Distance', 'Centroid Distance To Depot', 'Distance To Closest Other Cluster', 
                      'Distance To Closest Other Centroid', 'Cluster Area', 'Cluster Density', 'Savings', 'Marginal Costs']

    # Compute mean of all ratio_features (without the depot) and divide their values by mean to obtain ratio column which is inserted before the Shapley value
    for feature in ratio_features:
        mean = np.mean(instance_df.loc[1:, feature])
        ratio = instance_df.loc[1:, feature] / mean
        instance_df.insert(loc=len(instance_df.columns) - 3, column=feature + str(' Ratio'), value=ratio)

    if (prints == True):
        print('\n############### RATIO FEATURES ###############')
        print('No ratio features:\n', np.sort(list(set(instance_df.columns) - set(ratio_features) - set([i + str(' Ratio') for i in ratio_features]))))
        display(instance_df[instance_features + [i + str(' Ratio') for i in ratio_features]])

    # Drop depot row and reset index
    instance_df.drop(index=0, inplace=True)
    instance_df.reset_index(drop=True)

    # Merge instances
    df_final = pd.concat([df_final, instance_df], ignore_index=True)

    # Stop time count for instance size
    if (instance_id % number_of_instances_per_size == 0): run_time[num_customers] = time.time() - start

    # View final instance when there was only one modified instance
    if (max_instance_id == 1):
        print('\n############### FINAL INSTANCE ###############'); display(instance_df)
        plot_instance(coord=coord, sequence=None, total_costs=None, x_range=x_range, y_range=y_range, 
                 assignments=assignments, core_point_indices=core_point_indices, plot_sequence=True, print_sequence=False)

    # Else view run times and final DataFrame with merged instances
    elif (instance_id == list(data['Instance ID'])[-1]) or (instance_id == max_instance_id):
         print('-> Total run time: ', fun_convert_time(seconds=sum(run_time.values())))
         display({'Instance size: {} run time'.format(key): fun_convert_time(seconds=value) for key, value in run_time.items()})
         display(df_final)

    if (instance_id == max_instance_id): break

# Merge final DataFrame and DataFrame with previous cluster features from Johannes if parameter is True
if (old_cluster_features == True): df_final = pd.merge(left=df_final, right=df_clusters_johannes, how='left', left_index=True, right_index=True)

# Save file
fun_save_file(data=df_final, subfolder_path='..\\..\\01_data\\01_TSP', name='tsp_instances_j_updated.xlsx')

Processing: 100%|█████████▉| 8999/9000 [56:40<00:00,  2.80iteration/s]  

-> Total run time:  56m, 40s


{'Instance size: 6 run time': '4m, 21s',
 'Instance size: 7 run time': '6m, 18s',
 'Instance size: 8 run time': '6m, 48s',
 'Instance size: 9 run time': '6m, 46s',
 'Instance size: 10 run time': '6m, 58s',
 'Instance size: 11 run time': '7m, 12s',
 'Instance size: 12 run time': '5m, 48s',
 'Instance size: 13 run time': '6m, 27s',
 'Instance size: 14 run time': '5m, 58s'}

None

,Instance ID,Number Customers,X,Y,X Depot,Y Depot,Depot Distance,Closest Customer Distance (CCD),2nd CCD,3rd CCD,...,Centroid Distance To Depot Ratio,Distance To Closest Other Cluster Ratio,Distance To Closest Other Centroid Ratio,Cluster Area Ratio,Cluster Density Ratio,Savings Ratio,Marginal Costs Ratio,Shapley Value,SHAPO,Percentage Error
0,1,6,11.757432,50.848731,2.380844,66.016752,17.832253,29.874328,36.418450,42.990573,...,0.885295,1.199919,1.199889,1.199907,0.011512,0.022796,0.021584,6.805996,6.847093,0.006038
1,1,6,83.228495,41.537025,2.380844,66.016752,84.472480,33.588825,42.487567,51.532130,...,1.573524,0.000403,0.000554,0.000465,5.942439,2.161204,2.365312,73.361446,72.126416,0.016835
2,1,6,33.032921,29.876631,2.380844,66.016752,47.388376,9.140981,29.874328,32.182303,...,0.885295,1.199919,1.199889,1.199907,0.011512,0.186036,0.176144,21.568006,21.751330,0.008500
3,1,6,42.131509,30.755973,2.380844,66.016752,53.136032,9.140981,30.003533,30.228067,...,0.885295,1.199919,1.199889,1.199907,0.011512,0.514850,0.487476,25.980268,26.403603,0.016294
4,1,6,54.103013,58.267699,2.380844,66.016752,52.299433,30.003533,33.588825,35.355360,...,0.885295,1.199919,1.199889,1.199907,0.011512,0.101425,0.096032,30.191952,30.346193,0.005109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,9000,14,35.411268,13.512220,83.308855,83.076767,84.459488,7.012024,18.513792,29.590372,...,1.010867,1.166573,1.166526,1.166634,0.013872,0.368519,0.305413,32.946966,33.551630,0.018353
89996,9000,14,94.027698,4.059342,83.308855,83.076767,79.741125,42.666686,56.598097,58.952004,...,1.523562,0.000498,0.000877,0.000195,6.916770,4.429439,3.670935,70.014213,69.654750,0.005134
89997,9000,14,15.703090,62.406463,83.308855,83.076767,70.695127,23.369701,24.005740,25.154440,...,1.010867,1.166573,1.166526,1.166634,0.013872,2.593953,2.536474,37.349798,42.540654,0.138979
89998,9000,14,46.611090,57.482706,83.308855,83.076767,44.741277,16.222253,17.116747,18.088289,...,1.010867,1.166573,1.166526,1.166634,0.013872,0.163084,1.034829,16.911138,12.089902,0.285092


Processing: 100%|██████████| 9000/9000 [56:41<00:00,  2.65iteration/s]


File saved successfully!
